In [1]:
import pandas as pd
from scipy.io import arff
from treecombiner import TreeCombiner
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

In [4]:
DATASETS_COLUMNS = {
    'Height': 'childHeight'
} 

SEED = 14208

def process_dataset(filepath, extension, dataset_name):
    """
    Load and preprocess a dataset from a file.

    Parameters:
        filepath (str): The path to the dataset file.
        extension (str): The file extension (e.g., '.arff', '.csv').
        dataset_name (str): The name of the dataset.

    Returns:
        tuple: A tuple containing the training and validation data.
    """
    # Carga de datos
    if extension == '.arff':
        data = arff.loadarff(filepath)
        df = pd.DataFrame(data[0])
    elif extension == '.csv':
        df = pd.read_csv(filepath)
    else:
        raise ValueError("Unsupported file type. Use 'arff' or 'csv'.")

    # Preprocesamiento de datos
    train_df, validation_df = train_test_split(df, test_size=0.2, random_state=SEED)
    y_train, y_valid = train_df[DATASETS_COLUMNS[dataset_name]], validation_df[DATASETS_COLUMNS[dataset_name]]
    X_train, X_valid = train_df.drop(DATASETS_COLUMNS[dataset_name], axis=1), validation_df.drop(DATASETS_COLUMNS[dataset_name], axis=1)
    X_train = pd.get_dummies(X_train) # a chequear
    X_valid = pd.get_dummies(X_valid)
    X_train, X_valid = X_train.align(X_valid, join='left', axis=1, fill_value=0)

    return X_train, X_valid, y_train, y_valid

X_train, X_valid, y_train, y_valid = process_dataset('Height.csv', '.csv', 'Height')

In [5]:
rf = RandomForestRegressor(random_state=SEED, max_depth=1)
rf.fit(X_train, y_train)

RandomForestRegressor(max_depth=1, random_state=14208)

In [6]:
trees = rf.estimators_

In [12]:
samples_used = rf.estimators_samples_

samplesidx = set()
for samples in samples_used:
    for sampleidx in samples:
        samplesidx.add(sampleidx)
samplesidx = list(samplesidx)

# print(samplesidx)
def get_unions(X, y, samples_used_idx):
    return X[samples_used_idx], y[samples_used_idx]

X_union, y_union = get_unions(X_train, y_train, samplesidx)

KeyError: "None of [Index([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,\n       ...\n       737, 738, 739, 740, 741, 742, 743, 744, 745, 746],\n      dtype='int32', length=747)] are in the [columns]"